In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("twibot_tweet0_new.csv")
df

,author_id,created_at,in_reply_to_user_id,possibly_sensitive,retweet_count,like_count,text,label
0,3.726821e+08,2022-01-18 18:13:29+00:00,NaN,False,0,0,Same here..,human
1,3.726821e+08,2019-06-10 18:35:48+00:00,NaN,False,0,0,it's surprising how some people take advantage...,human
2,6.252442e+07,2010-09-03 12:43:53+00:00,NaN,False,0,0,Hey guys wassup..??,human
3,6.252442e+07,2010-08-24 13:07:34+00:00,NaN,False,0,0,Struk in traffic,human
4,6.252442e+07,2010-08-13 12:56:55+00:00,NaN,False,0,0,Live life love life.,human
...,...,...,...,...,...,...,...,...
9995,2.385720e+08,2022-02-27 17:06:26+00:00,NaN,False,0,1,Wars are always heartbreaking and nothing good...,human
9996,8.098110e+17,2022-01-04 12:19:11+00:00,NaN,False,0,0,Let's see how much time it's gonna take,human
9997,2.065347e+07,2021-12-29 17:45:12+00:00,NaN,False,0,3,"Compassion requires us to care, to be empathet...",human
9998,1.191930e+18,2020-11-04 20:54:32+00:00,1.191930e+18,False,0,9,Let us continue the good fight.\nLet us make a...,human


In [5]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# XGBoost モデルの学習
xgb_model = XGBClassifier(n_estimators=100, random_state=42, use_label_encoder=False, eval_metric="logloss")
xgb_model.fit(X_train_smote, y_train_smote)

# 予測と評価
y_pred_xgb = xgb_model.predict(X_test_scaled)
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
report_xgb = classification_report(y_test, y_pred_xgb)

print("Accuracy:", accuracy_xgb)
print("Classification Report:\n", report_xgb)


NameError: name 'X_train_smote' is not defined

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

# データの読み込み
df = pd.read_csv('twibot_tweet0_new.csv')

# テキストとラベルの抽出
texts = df['text'].tolist()
labels = df['label'].map({'human': 0, 'bot': 1}).tolist()

# データ分割 (トレーニングセットとテストセット)
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# テキストデータをベクトル化
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# オーバーサンプリング (SMOTE)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)

# ナイーブベイズモデルの訓練
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_resampled, y_train_resampled)

# テストデータでの予測
y_pred = nb_classifier.predict(X_test_vectorized)

# モデル評価
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['human', 'bot']))


Accuracy: 0.8210

Classification Report:
              precision    recall  f1-score   support

       human       0.90      0.90      0.90      1793
         bot       0.11      0.10      0.11       207

    accuracy                           0.82      2000
   macro avg       0.50      0.50      0.50      2000
weighted avg       0.82      0.82      0.82      2000



In [3]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm  # 進捗バーのライブラリ

# データの読み込みと前処理
df = pd.read_csv('twibot_tweet0_new.csv')
texts = df['text'].tolist()
labels = df['label'].map({'human': 0, 'bot': 1}).tolist()

# データの分割
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# BERTモデルとトークナイザーの準備
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# データセットクラスの定義
class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, item):
        encoding = self.tokenizer.encode_plus(
            self.texts[item],
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[item], dtype=torch.long)
        }

# データローダーの作成
train_dataset = TweetDataset(train_texts, train_labels, tokenizer, max_len=128)
val_dataset = TweetDataset(val_texts, val_labels, tokenizer, max_len=128)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# モデルの学習
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)

# 学習ループに進捗バーを追加
epochs = 3  # エポック数

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    model.train()
    train_loss = 0
    
    # tqdmで進捗バーを表示
    for batch in tqdm(train_loader, desc="Training"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    
    print(f"Training Loss: {train_loss / len(train_loader):.4f}")
    
    # バリデーション
    model.eval()
    val_accuracy = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            val_accuracy += torch.sum(preds == labels).item()
    
    val_accuracy /= len(val_dataset)
    print(f"Validation Accuracy: {val_accuracy:.4f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3


Training: 100%|███████████████████████████████████████████████████████████████████████████████████| 500/500 [31:49<00:00,  3.82s/it]


Training Loss: 0.3328


Validation: 100%|█████████████████████████████████████████████████████████████████████████████████| 125/125 [02:23<00:00,  1.15s/it]


...

Validation Accuracy: 0.8950
Epoch 3/3


Training: 100%|███████████████████████████████████████████████████████████████████████████████████| 500/500 [36:17<00:00,  4.35s/it]


Training Loss: 0.2107


Validation: 100%|█████████████████████████████████████████████████████████████████████████████████| 125/125 [02:27<00:00,  1.18s/it]

Validation Accuracy: 0.8910


In [4]:
from sklearn.metrics import classification_report

# バリデーションデータでの予測
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# クラスごとの評価
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=['human', 'bot']))


Classification Report:
              precision    recall  f1-score   support

       human       0.90      0.99      0.94      1793
         bot       0.31      0.04      0.08       207

    accuracy                           0.89      2000
   macro avg       0.60      0.52      0.51      2000
weighted avg       0.84      0.89      0.85      2000



In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

# データの読み込み
df = pd.read_csv('twibot_tweet0_new.csv')

# テキストとラベルの抽出
texts = df['text'].tolist()
labels = df['label'].map({'human': 0, 'bot': 1}).tolist()

# データ分割
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# テキストデータをベクトル化
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# SMOTEを使用してオーバーサンプリング
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)

# ナイーブベイズモデルの訓練
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_resampled, y_train_resampled)

# テストデータでの予測
y_pred = nb_classifier.predict(X_test_vectorized)

# モデル評価
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['human', 'bot']))


Accuracy: 0.8210

Classification Report:
              precision    recall  f1-score   support

       human       0.90      0.90      0.90      1793
         bot       0.11      0.10      0.11       207

    accuracy                           0.82      2000
   macro avg       0.50      0.50      0.50      2000
weighted avg       0.82      0.82      0.82      2000



In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# データの読み込みと前処理
df = pd.read_csv('twibot_tweet0_new.csv')
texts = df['text'].tolist()
labels = df['label'].map({'human': 0, 'bot': 1}).tolist()

# データ分割
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# テキストデータをベクトル化
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# SVMモデルの訓練
svm_classifier = SVC(kernel='rbf', C=1.0, random_state=42)
svm_classifier.fit(X_train_vectorized, y_train)

# テストデータでの予測
y_pred = svm_classifier.predict(X_test_vectorized)

# モデル評価
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['human', 'bot']))


Accuracy: 0.8970

Classification Report:
              precision    recall  f1-score   support

       human       0.90      1.00      0.95      1793
         bot       1.00      0.00      0.01       207

    accuracy                           0.90      2000
   macro avg       0.95      0.50      0.48      2000
weighted avg       0.91      0.90      0.85      2000



In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

# データの読み込みと前処理
df = pd.read_csv('twibot_tweet0_new.csv')
texts = df['text'].tolist()
labels = df['label'].map({'human': 0, 'bot': 1}).tolist()

# データ分割
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# テキストデータをベクトル化
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# SMOTEを使用してオーバーサンプリング
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)

# SVMモデルの訓練
svm_classifier = SVC(kernel='rbf', C=1.0, random_state=42)
svm_classifier.fit(X_train_resampled, y_train_resampled)

# テストデータでの予測
y_pred = svm_classifier.predict(X_test_vectorized)

# モデル評価
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['human', 'bot']))


Accuracy: 0.8935

Classification Report:
              precision    recall  f1-score   support

       human       0.90      0.99      0.94      1793
         bot       0.38      0.05      0.09       207

    accuracy                           0.89      2000
   macro avg       0.64      0.52      0.51      2000
weighted avg       0.85      0.89      0.85      2000



In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

# データの読み込みと前処理
df = pd.read_csv('twibot_tweet0_new.csv')
texts = df['text'].tolist()
labels = df['label'].map({'human': 0, 'bot': 1}).tolist()

# データ分割
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# テキストデータをベクトル化（TF-IDF）
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# SMOTEを使用してオーバーサンプリング
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)

# ランダムフォレストモデルの訓練
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_resampled, y_train_resampled)

# テストデータでの予測
y_pred = rf_classifier.predict(X_test_vectorized)

# モデル評価
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['human', 'bot']))


Accuracy: 0.8845

Classification Report:
              precision    recall  f1-score   support

       human       0.90      0.98      0.94      1793
         bot       0.12      0.02      0.03       207

    accuracy                           0.88      2000
   macro avg       0.51      0.50      0.49      2000
weighted avg       0.82      0.88      0.84      2000



In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

# データの読み込みと前処理
df = pd.read_csv('twibot_tweet0_new.csv')
texts = df['text'].tolist()
labels = df['label'].map({'human': 0, 'bot': 1}).tolist()

# データ分割
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# テキストデータをベクトル化（TF-IDF）
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# SMOTEを使用してオーバーサンプリング
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)

# XGBoostモデルの構築と訓練
xgb_classifier = XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)
xgb_classifier.fit(X_train_resampled, y_train_resampled)

# テストデータでの予測
y_pred = xgb_classifier.predict(X_test_vectorized)

# モデル評価
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['human', 'bot']))


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:40:14] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.8125

Classification Report:
              precision    recall  f1-score   support

       human       0.90      0.89      0.90      1793
         bot       0.11      0.12      0.11       207

    accuracy                           0.81      2000
   macro avg       0.50      0.50      0.50      2000
weighted avg       0.82      0.81      0.81      2000



In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import OneClassSVM
from sklearn.metrics import classification_report
from scipy.sparse import csr_matrix

# データの読み込みと前処理
df = pd.read_csv('twibot_tweet0_new.csv')
X = df['text']
y = df['label'].map({'human': 0, 'bot': 1})

# テキストデータをTF-IDFベクトルに変換
vectorizer = TfidfVectorizer(max_features=1000)
X_vectorized = vectorizer.fit_transform(X)

# データを訓練セットとテストセットに分割
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

# One-Class SVMモデルの訓練（humanデータのみを使用）
human_train = X_train[y_train == 0]
human_train_csr = csr_matrix(human_train)

svm = OneClassSVM(kernel='rbf', nu=0.1)
svm.fit(human_train_csr)

# テストデータに対する予測
X_test_csr = csr_matrix(X_test)
y_pred = svm.predict(X_test_csr)
y_pred = [1 if x == -1 else 0 for x in y_pred]  # -1を1（異常）、1を0（正常）に変換

# 結果の評価
print(classification_report(y_test, y_pred, target_names=['human', 'bot']))


              precision    recall  f1-score   support

       human       0.90      0.82      0.86      1793
         bot       0.11      0.19      0.14       207

    accuracy                           0.76      2000
   macro avg       0.50      0.50      0.50      2000
weighted avg       0.82      0.76      0.78      2000



In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# データの読み込みと前処理
df = pd.read_csv('twibot_tweet0_new.csv')
X = df['text']
y = df['label'].map({'human': 0, 'bot': 1})

# テキストデータをTF-IDFベクトルに変換
vectorizer = TfidfVectorizer(max_features=1000)
X_vectorized = vectorizer.fit_transform(X)

# データを訓練セットとテストセットに分割
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

# LOFモデルの構築と訓練（humanデータのみを使用）
human_train = X_train[y_train == 0]
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.1, novelty=True)
lof.fit(human_train)

# テストデータに対する予測
y_pred = lof.predict(X_test)
y_pred = [1 if x == -1 else 0 for x in y_pred]  # -1をbot(1)、1をhuman(0)に変換

# クラス数確認（デバッグ用）
print("y_test unique values:", set(y_test))
print("y_pred unique values:", set(y_pred))

# 結果の評価
print(classification_report(y_test, y_pred, target_names=['human', 'bot']))


y_test unique values: {0, 1}
y_pred unique values: {0, 1}
              precision    recall  f1-score   support

       human       0.90      0.97      0.93      1793
         bot       0.05      0.01      0.02       207

    accuracy                           0.87      2000
   macro avg       0.47      0.49      0.48      2000
weighted avg       0.81      0.87      0.84      2000



In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.metrics import classification_report

# データの読み込みと前処理
df = pd.read_csv('twibot_tweet0_new.csv')
X = df['text']
y = df['label'].map({'human': 0, 'bot': 1})

# テキストデータをTF-IDFベクトルに変換
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1000)
X_vectorized = vectorizer.fit_transform(X).toarray()

# データを訓練セットとテストセットに分割
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

# データの正規化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Autoencoderモデルの構築
input_dim = X_train_scaled.shape[1]
encoding_dim = 32

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="relu")(input_layer)
decoder = Dense(input_dim, activation="sigmoid")(encoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

autoencoder.compile(optimizer='adam', loss='mse')

# モデルの訓練（正常データのみを使用）
normal_X_train = X_train_scaled[y_train == 0]
autoencoder.fit(normal_X_train, normal_X_train, epochs=50, batch_size=32, shuffle=True, validation_split=0.1, verbose=0)

# 再構成誤差の計算
mse = np.mean(np.power(X_test_scaled - autoencoder.predict(X_test_scaled), 2), axis=1)

# しきい値の設定（例：95パーセンタイル）
threshold = np.percentile(mse, 95)

# 異常検知
y_pred = [1 if error > threshold else 0 for error in mse]

# 結果の評価
print(classification_report(y_test, y_pred, target_names=['human', 'bot']))


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step
              precision    recall  f1-score   support

       human       0.90      0.95      0.92      1793
         bot       0.11      0.05      0.07       207

    accuracy                           0.86      2000
   macro avg       0.50      0.50      0.50      2000
weighted avg       0.82      0.86      0.83      2000



In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from scipy.sparse import csr_matrix

# データの読み込みと前処理
df = pd.read_csv('twibot_tweet0_new.csv')
X = df['text']
y = df['label'].map({'human': 0, 'bot': 1})

# テキストデータをTF-IDFベクトルに変換
vectorizer = TfidfVectorizer(max_features=1000)
X_vectorized = vectorizer.fit_transform(X)

# データを訓練セットとテストセットに分割
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

# 正常データ（humanのみ）を抽出
X_train_normal = X_train[y_train == 0]
X_train_normal = csr_matrix(X_train_normal)

# One-Class SVM
ocsvm = OneClassSVM(kernel='rbf', nu=0.1)
ocsvm.fit(X_train_normal)

# Local Outlier Factor
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.1, novelty=True)
lof.fit(X_train_normal)

# Autoencoder
scaler = StandardScaler()
X_train_normal_dense = scaler.fit_transform(X_train_normal.toarray())

input_dim = X_train_normal_dense.shape[1]
encoding_dim = 32

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="relu")(input_layer)
decoder = Dense(input_dim, activation="sigmoid")(encoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(X_train_normal_dense, X_train_normal_dense, epochs=50, batch_size=32, shuffle=True, validation_split=0.1, verbose=0)

# テストデータに対する予測
X_test_csr = csr_matrix(X_test)
X_test_dense = scaler.transform(X_test.toarray())

y_pred_ocsvm = ocsvm.predict(X_test_csr)
y_pred_lof = lof.predict(X_test_csr)
mse = np.mean(np.power(X_test_dense - autoencoder.predict(X_test_dense), 2), axis=1)
threshold = np.percentile(mse, 90)
y_pred_autoencoder = [1 if error > threshold else 0 for error in mse]

# アンサンブル予測（多数決）
y_pred_ensemble = np.mean([y_pred_ocsvm, y_pred_lof, y_pred_autoencoder], axis=0)
y_pred_ensemble = [1 if x <= -0.5 else 0 for x in y_pred_ensemble]

# 結果の評価
print(classification_report(y_test, y_pred_ensemble, target_names=['human', 'bot']))


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 989us/step
              precision    recall  f1-score   support

       human       0.90      1.00      0.95      1793
         bot       0.00      0.00      0.00       207

    accuracy                           0.90      2000
   macro avg       0.45      0.50      0.47      2000
weighted avg       0.80      0.90      0.85      2000



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

In [28]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn.metrics import classification_report
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split

# データの読み込みと前処理
df = pd.read_csv('twibot_tweet0_new.csv')
X_text = df['text']
X_numeric = df[['retweet_count', 'like_count']]
y = df['label'].map({'human': 0, 'bot': 1})

# テキストデータをTF-IDFベクトルに変換
vectorizer = TfidfVectorizer(max_features=1000)
X_text_vectorized = vectorizer.fit_transform(X_text)

# 数値データの正規化
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

# テキスト特徴量と数値特徴量を結合し、csr_matrix形式に変換
X_combined = hstack([X_text_vectorized, X_numeric_scaled])
X_combined = csr_matrix(X_combined)

# データを訓練セットとテストセットに分割
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# 正常データ（humanのみ）を抽出
X_train_normal = X_train[np.array(y_train == 0).flatten()]

# One-Class SVMモデルの構築と学習
svm = OneClassSVM(kernel='rbf', nu=0.1)
svm.fit(X_train_normal)

# テストデータに対する予測
y_pred = svm.predict(X_test)
y_pred = [1 if x == -1 else 0 for x in y_pred]  # -1をbot(1)、1をhuman(0)に変換

# 結果の評価
print(classification_report(y_test, y_pred, target_names=['human', 'bot']))

              precision    recall  f1-score   support

       human       0.90      0.87      0.88      1793
         bot       0.10      0.13      0.12       207

    accuracy                           0.79      2000
   macro avg       0.50      0.50      0.50      2000
weighted avg       0.81      0.79      0.80      2000



In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from scipy.sparse import hstack

# データの読み込みと前処理
df = pd.read_csv('twibot_tweet0_new.csv')

# 特徴量エンジニアリング
df['text_length'] = df['text'].str.len()
df['has_url'] = df['text'].str.contains('http').astype(int)
df['hour'] = pd.to_datetime(df['created_at']).dt.hour
df['day_of_week'] = pd.to_datetime(df['created_at']).dt.dayofweek
df['retweet_like_ratio'] = df['retweet_count'] / (df['like_count'] + 1)

# テキストデータをTF-IDFベクトルに変換
vectorizer = TfidfVectorizer(max_features=1000)
X_text = vectorizer.fit_transform(df['text'])

# 数値特徴量の準備
numeric_features = ['retweet_count', 'like_count', 'text_length', 'has_url', 'hour', 'day_of_week', 'retweet_like_ratio']
X_numeric = df[numeric_features]

# 数値データの正規化
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

# 特徴量の結合
X_combined = hstack([X_text, X_numeric_scaled])

# ラベルの準備
y = df['label'].map({'human': 0, 'bot': 1})

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# モデルの構築と学習
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# 予測と評価
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['human', 'bot']))


              precision    recall  f1-score   support

       human       0.90      1.00      0.94      1793
         bot       0.36      0.02      0.04       207

    accuracy                           0.90      2000
   macro avg       0.63      0.51      0.49      2000
weighted avg       0.84      0.90      0.85      2000



In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from scipy.sparse import hstack
import re

# データの読み込みと前処理
df = pd.read_csv('twibot_tweet0_new.csv')

# 特徴量エンジニアリング
df['text_length'] = df['text'].str.len()
df['has_url'] = df['text'].str.contains('http').astype(int)
df['hour'] = pd.to_datetime(df['created_at']).dt.hour
df['day_of_week'] = pd.to_datetime(df['created_at']).dt.dayofweek
df['is_reply'] = df['in_reply_to_user_id'].notna().astype(int)
df['hashtag_count'] = df['text'].apply(lambda x: len(re.findall(r'#\w+', x)))
df['mention_count'] = df['text'].apply(lambda x: len(re.findall(r'@\w+', x)))
df['retweet_like_ratio'] = df['retweet_count'] / (df['like_count'] + 1)

# テキストデータをTF-IDFベクトルに変換
vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_text = vectorizer.fit_transform(df['text'])

# 数値特徴量の準備
numeric_features = ['retweet_count', 'like_count', 'text_length', 'has_url', 'hour', 'day_of_week', 'is_reply', 'hashtag_count', 'mention_count', 'retweet_like_ratio']
X_numeric = df[numeric_features]

# 数値データの正規化
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

# 特徴量の結合
X_combined = hstack([X_text, X_numeric_scaled])

# ラベルの準備
y = df['label'].map({'human': 0, 'bot': 1})

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42, stratify=y)

# クラスの重みを調整
class_weights = {0: 1, 1: 5}  # botクラスの重みを増やす

# モデルの構築と学習
clf = RandomForestClassifier(n_estimators=200, class_weight=class_weights, random_state=42)
clf.fit(X_train, y_train)

# 予測と評価
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['human', 'bot']))


              precision    recall  f1-score   support

       human       0.90      1.00      0.95      1796
         bot       0.38      0.01      0.03       204

    accuracy                           0.90      2000
   macro avg       0.64      0.51      0.49      2000
weighted avg       0.85      0.90      0.85      2000



In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from scipy.sparse import hstack
from imblearn.over_sampling import SMOTE
import re
from textblob import TextBlob

# データの読み込みと前処理
df = pd.read_csv('twibot_tweet0_new.csv')

# 特徴量エンジニアリング
def extract_features(df):
    df['text_length'] = df['text'].str.len()
    df['has_url'] = df['text'].str.contains('http').astype(int)
    df['hour'] = pd.to_datetime(df['created_at']).dt.hour
    df['day_of_week'] = pd.to_datetime(df['created_at']).dt.dayofweek
    df['is_reply'] = df['in_reply_to_user_id'].notna().astype(int)
    df['hashtag_count'] = df['text'].apply(lambda x: len(re.findall(r'#\w+', x)))
    df['mention_count'] = df['text'].apply(lambda x: len(re.findall(r'@\w+', x)))
    df['retweet_like_ratio'] = df['retweet_count'] / (df['like_count'] + 1)
    df['sentiment'] = df['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
    df['subjectivity'] = df['text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    return df

df = extract_features(df)

# テキストデータをTF-IDFベクトルに変換
vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_text = vectorizer.fit_transform(df['text'])

# 数値特徴量の準備
numeric_features = ['retweet_count', 'like_count', 'text_length', 'has_url', 'hour', 'day_of_week', 'is_reply', 'hashtag_count', 'mention_count', 'retweet_like_ratio', 'sentiment', 'subjectivity']
X_numeric = df[numeric_features]

# 数値データの正規化
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

# 特徴量の結合
X_combined = hstack([X_text, X_numeric_scaled])

# ラベルの準備
y = df['label'].map({'human': 0, 'bot': 1})

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42, stratify=y)

# SMOTEを適用
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# モデルの構築
rf = RandomForestClassifier(n_estimators=200, random_state=42)
gb = GradientBoostingClassifier(n_estimators=200, random_state=42)
svm = SVC(kernel='rbf', probability=True, random_state=42)

# モデルの学習
rf.fit(X_train_resampled, y_train_resampled)
gb.fit(X_train_resampled, y_train_resampled)
svm.fit(X_train_resampled, y_train_resampled)

# アンサンブル予測
y_pred_rf = rf.predict_proba(X_test)[:, 1]
y_pred_gb = gb.predict_proba(X_test)[:, 1]
y_pred_svm = svm.predict_proba(X_test)[:, 1]

y_pred_ensemble = (y_pred_rf + y_pred_gb + y_pred_svm) / 3
y_pred_final = (y_pred_ensemble > 0.5).astype(int)

# 結果の評価
print(classification_report(y_test, y_pred_final, target_names=['human', 'bot']))


              precision    recall  f1-score   support

       human       0.90      0.97      0.93      1796
         bot       0.21      0.08      0.11       204

    accuracy                           0.88      2000
   macro avg       0.56      0.52      0.52      2000
weighted avg       0.83      0.88      0.85      2000



In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from scipy.sparse import hstack
from imblearn.over_sampling import SMOTE
import re
from textblob import TextBlob

def extract_features(df):
    df['text_length'] = df['text'].str.len()
    df['has_url'] = df['text'].str.contains('http').astype(int)
    df['hour'] = pd.to_datetime(df['created_at']).dt.hour
    df['day_of_week'] = pd.to_datetime(df['created_at']).dt.dayofweek
    df['is_reply'] = df['in_reply_to_user_id'].notna().astype(int)
    df['hashtag_count'] = df['text'].apply(lambda x: len(re.findall(r'#\w+', x)))
    df['mention_count'] = df['text'].apply(lambda x: len(re.findall(r'@\w+', x)))
    df['retweet_like_ratio'] = df['retweet_count'] / (df['like_count'] + 1)
    df['sentiment'] = df['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
    df['subjectivity'] = df['text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    df['caps_ratio'] = df['text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)
    df['unique_words_ratio'] = df['text'].apply(lambda x: len(set(x.split())) / len(x.split()) if len(x.split()) > 0 else 0)
    return df

df = pd.read_csv('twibot_tweet0_new.csv')
df = extract_features(df)

vectorizer = TfidfVectorizer(max_features=1000, stop_words='english', ngram_range=(1, 2))
X_text = vectorizer.fit_transform(df['text'])

numeric_features = ['retweet_count', 'like_count', 'text_length', 'has_url', 'hour', 'day_of_week', 'is_reply', 'hashtag_count', 'mention_count', 'retweet_like_ratio', 'sentiment', 'subjectivity', 'caps_ratio', 'unique_words_ratio']
X_numeric = df[numeric_features]

scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

X_combined = hstack([X_text, X_numeric_scaled])
y = df['label'].map({'human': 0, 'bot': 1})

X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42, stratify=y)

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

rf = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42)
gb = GradientBoostingClassifier(n_estimators=200, random_state=42)
svm = SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=42)

rf.fit(X_train_resampled, y_train_resampled)
gb.fit(X_train_resampled, y_train_resampled)
svm.fit(X_train_resampled, y_train_resampled)

y_pred_rf = rf.predict_proba(X_test)[:, 1]
y_pred_gb = gb.predict_proba(X_test)[:, 1]
y_pred_svm = svm.predict_proba(X_test)[:, 1]

y_pred_ensemble = 0.4 * y_pred_rf + 0.4 * y_pred_gb + 0.2 * y_pred_svm
y_pred_final = (y_pred_ensemble > 0.4).astype(int)

print(classification_report(y_test, y_pred_final, target_names=['human', 'bot']))


              precision    recall  f1-score   support

       human       0.91      0.93      0.92      1796
         bot       0.19      0.15      0.17       204

    accuracy                           0.85      2000
   macro avg       0.55      0.54      0.54      2000
weighted avg       0.83      0.85      0.84      2000



In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from scipy.sparse import hstack
from imblearn.over_sampling import SMOTE
import re
from textblob import TextBlob

def extract_features(df):
    df['text_length'] = df['text'].str.len()
    df['has_url'] = df['text'].str.contains('http').astype(int)
    df['hour'] = pd.to_datetime(df['created_at']).dt.hour
    df['day_of_week'] = pd.to_datetime(df['created_at']).dt.dayofweek
    df['is_reply'] = df['in_reply_to_user_id'].notna().astype(int)
    df['hashtag_count'] = df['text'].apply(lambda x: len(re.findall(r'#\w+', x)))
    df['mention_count'] = df['text'].apply(lambda x: len(re.findall(r'@\w+', x)))
    df['retweet_like_ratio'] = df['retweet_count'] / (df['like_count'] + 1)
    df['sentiment'] = df['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
    df['subjectivity'] = df['text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    df['caps_ratio'] = df['text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)
    df['unique_words_ratio'] = df['text'].apply(lambda x: len(set(x.split())) / len(x.split()) if len(x.split()) > 0 else 0)
    df['tweet_frequency'] = df.groupby('author_id')['created_at'].transform('count')
    return df

df = pd.read_csv('twibot_tweet0_new.csv')
df = extract_features(df)

X_text = df['text']
numeric_features = ['retweet_count', 'like_count', 'text_length', 'has_url', 'hour', 'day_of_week', 'is_reply', 'hashtag_count', 'mention_count', 'retweet_like_ratio', 'sentiment', 'subjectivity', 'caps_ratio', 'unique_words_ratio', 'tweet_frequency']
X_numeric = df[numeric_features]
y = df['label'].map({'human': 0, 'bot': 1})

X_train_text, X_test_text, X_train_numeric, X_test_numeric, y_train, y_test = train_test_split(X_text, X_numeric, y, test_size=0.2, random_state=42, stratify=y)

tfidf = TfidfVectorizer(max_features=2000, ngram_range=(1, 3))
X_train_tfidf = tfidf.fit_transform(X_train_text)
X_test_tfidf = tfidf.transform(X_test_text)

scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.transform(X_test_numeric)

X_train_combined = hstack([X_train_tfidf, X_train_numeric_scaled])
X_test_combined = hstack([X_test_tfidf, X_test_numeric_scaled])

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_combined, y_train)

rf = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42)
gb = GradientBoostingClassifier(n_estimators=200, random_state=42)
svm = SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=42)

rf.fit(X_train_resampled, y_train_resampled)
gb.fit(X_train_resampled, y_train_resampled)
svm.fit(X_train_resampled, y_train_resampled)

y_pred_rf = rf.predict_proba(X_test_combined)[:, 1]
y_pred_gb = gb.predict_proba(X_test_combined)[:, 1]
y_pred_svm = svm.predict_proba(X_test_combined)[:, 1]

y_pred_ensemble = 0.4 * y_pred_rf + 0.4 * y_pred_gb + 0.2 * y_pred_svm
y_pred_final = (y_pred_ensemble > 0.4).astype(int)

print(classification_report(y_test, y_pred_final, target_names=['human', 'bot']))


              precision    recall  f1-score   support

       human       0.92      0.94      0.93      1796
         bot       0.35      0.26      0.30       204

    accuracy                           0.88      2000
   macro avg       0.63      0.60      0.61      2000
weighted avg       0.86      0.88      0.87      2000



In [35]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from scipy.sparse import hstack
from imblearn.over_sampling import SMOTE
import re
from textblob import TextBlob
from tqdm import tqdm
import time

def extract_features(df):
    df['text_length'] = df['text'].str.len()
    df['has_url'] = df['text'].str.contains('http').astype(int)
    df['hour'] = pd.to_datetime(df['created_at']).dt.hour
    df['day_of_week'] = pd.to_datetime(df['created_at']).dt.dayofweek
    df['is_reply'] = df['in_reply_to_user_id'].notna().astype(int)
    df['hashtag_count'] = df['text'].apply(lambda x: len(re.findall(r'#\w+', x)))
    df['mention_count'] = df['text'].apply(lambda x: len(re.findall(r'@\w+', x)))
    df['retweet_like_ratio'] = df['retweet_count'] / (df['like_count'] + 1)
    df['sentiment'] = df['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
    df['subjectivity'] = df['text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    df['caps_ratio'] = df['text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)
    df['unique_words_ratio'] = df['text'].apply(lambda x: len(set(x.split())) / len(x.split()) if len(x.split()) > 0 else 0)
    df['tweet_frequency'] = df.groupby('author_id')['created_at'].transform('count')
    return df

print("データの読み込みと前処理を開始します...")
df = pd.read_csv('twibot_tweet0_new.csv')
df = extract_features(df)
print("前処理が完了しました。")

X_text = df['text']
numeric_features = ['retweet_count', 'like_count', 'text_length', 'has_url', 'hour', 'day_of_week', 'is_reply', 'hashtag_count', 'mention_count', 'retweet_like_ratio', 'sentiment', 'subjectivity', 'caps_ratio', 'unique_words_ratio', 'tweet_frequency']
X_numeric = df[numeric_features]
y = df['label'].map({'human': 0, 'bot': 1})

print("データの分割を行います...")
X_train_text, X_test_text, X_train_numeric, X_test_numeric, y_train, y_test = train_test_split(X_text, X_numeric, y, test_size=0.2, random_state=42, stratify=y)

print("テキストデータの変換を開始します...")
tfidf = TfidfVectorizer(max_features=3000, ngram_range=(1, 3))
X_train_tfidf = tfidf.fit_transform(X_train_text)
X_test_tfidf = tfidf.transform(X_test_text)
print("テキストデータの変換が完了しました。")

print("数値データの正規化を行います...")
scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.transform(X_test_numeric)

X_train_combined = hstack([X_train_tfidf, X_train_numeric_scaled])
X_test_combined = hstack([X_test_tfidf, X_test_numeric_scaled])

print("SMOTEを適用してデータのバランスを調整します...")
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_combined, y_train)
print("データのバランス調整が完了しました。")

models = {
    'RandomForest': RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=200, random_state=42),
    'SVM': SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=42)
}

print("モデルの学習を開始します...")
for name, model in tqdm(models.items()):
    print(f"{name}の学習を開始します...")
    start_time = time.time()
    model.fit(X_train_resampled, y_train_resampled)
    end_time = time.time()
    print(f"{name}の学習が完了しました。学習時間: {end_time - start_time:.2f}秒")

print("アンサンブル予測を行います...")
y_pred_rf = models['RandomForest'].predict_proba(X_test_combined)[:, 1]
y_pred_gb = models['GradientBoosting'].predict_proba(X_test_combined)[:, 1]
y_pred_svm = models['SVM'].predict_proba(X_test_combined)[:, 1]

y_pred_ensemble = 0.4 * y_pred_rf + 0.4 * y_pred_gb + 0.2 * y_pred_svm
y_pred_final = (y_pred_ensemble > 0.4).astype(int)

print("最終的な予測結果:")
print(classification_report(y_test, y_pred_final, target_names=['human', 'bot']))


データの読み込みと前処理を開始します...
前処理が完了しました。
データの分割を行います...
テキストデータの変換を開始します...
テキストデータの変換が完了しました。
数値データの正規化を行います...
SMOTEを適用してデータのバランスを調整します...
データのバランス調整が完了しました。
モデルの学習を開始します...


  0%|                                                                                                         | 0/3 [00:00<?, ?it/s]

RandomForestの学習を開始します...


 33%|████████████████████████████████▎                                                                | 1/3 [00:10<00:21, 10.85s/it]

RandomForestの学習が完了しました。学習時間: 10.85秒
GradientBoostingの学習を開始します...


 67%|████████████████████████████████████████████████████████████████▋                                | 2/3 [00:43<00:23, 23.88s/it]

GradientBoostingの学習が完了しました。学習時間: 33.00秒
SVMの学習を開始します...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [03:36<00:00, 72.19s/it]

SVMの学習が完了しました。学習時間: 172.73秒
アンサンブル予測を行います...


最終的な予測結果:
              precision    recall  f1-score   support

       human       0.92      0.95      0.93      1796
         bot       0.35      0.25      0.29       204

    accuracy                           0.88      2000
   macro avg       0.63      0.60      0.61      2000
weighted avg       0.86      0.88      0.87      2000



In [36]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from scipy.sparse import hstack
from imblearn.over_sampling import SMOTE
import re
from textblob import TextBlob
from tqdm import tqdm
import time
from sklearn.neural_network import MLPClassifier

def extract_features(df):
    df['text_length'] = df['text'].str.len()
    df['has_url'] = df['text'].str.contains('http').astype(int)
    df['hour'] = pd.to_datetime(df['created_at']).dt.hour
    df['day_of_week'] = pd.to_datetime(df['created_at']).dt.dayofweek
    df['is_reply'] = df['in_reply_to_user_id'].notna().astype(int)
    df['hashtag_count'] = df['text'].apply(lambda x: len(re.findall(r'#\w+', x)))
    df['mention_count'] = df['text'].apply(lambda x: len(re.findall(r'@\w+', x)))
    df['retweet_like_ratio'] = df['retweet_count'] / (df['like_count'] + 1)
    df['sentiment'] = df['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
    df['subjectivity'] = df['text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    df['caps_ratio'] = df['text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)
    df['unique_words_ratio'] = df['text'].apply(lambda x: len(set(x.split())) / len(x.split()) if len(x.split()) > 0 else 0)
    df['tweet_frequency'] = df.groupby('author_id')['created_at'].transform('count')
    return df

print("データの読み込みと前処理を開始します...")
df = pd.read_csv('twibot_tweet0_new.csv')
df = extract_features(df)
print("前処理が完了しました。")

X_text = df['text']
numeric_features = ['retweet_count', 'like_count', 'text_length', 'has_url', 'hour', 'day_of_week', 'is_reply', 'hashtag_count', 'mention_count', 'retweet_like_ratio', 'sentiment', 'subjectivity', 'caps_ratio', 'unique_words_ratio', 'tweet_frequency']
X_numeric = df[numeric_features]
y = df['label'].map({'human': 0, 'bot': 1})

print("データの分割を行います...")
X_train_text, X_test_text, X_train_numeric, X_test_numeric, y_train, y_test = train_test_split(X_text, X_numeric, y, test_size=0.2, random_state=42, stratify=y)

print("テキストデータの変換を開始します...")
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 3))
X_train_tfidf = tfidf.fit_transform(X_train_text)
X_test_tfidf = tfidf.transform(X_test_text)
print("テキストデータの変換が完了しました。")

print("数値データの正規化を行います...")
scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.transform(X_test_numeric)

X_train_combined = hstack([X_train_tfidf, X_train_numeric_scaled])
X_test_combined = hstack([X_test_tfidf, X_test_numeric_scaled])

print("SMOTEを適用してデータのバランスを調整します...")
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_combined, y_train)
print("データのバランス調整が完了しました。")

models = {
    'RandomForest': RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=200, random_state=42),
    'SVM': SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=42),
    'MLP': MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=300, random_state=42)
}

print("モデルの学習を開始します...")
for name, model in tqdm(models.items()):
    print(f"{name}の学習を開始します...")
    start_time = time.time()
    model.fit(X_train_resampled, y_train_resampled)
    end_time = time.time()
    print(f"{name}の学習が完了しました。学習時間: {end_time - start_time:.2f}秒")

print("アンサンブル予測を行います...")
y_pred_rf = models['RandomForest'].predict_proba(X_test_combined)[:, 1]
y_pred_gb = models['GradientBoosting'].predict_proba(X_test_combined)[:, 1]
y_pred_svm = models['SVM'].predict_proba(X_test_combined)[:, 1]
y_pred_mlp = models['MLP'].predict_proba(X_test_combined)[:, 1]

y_pred_ensemble = 0.3 * y_pred_rf + 0.3 * y_pred_gb + 0.2 * y_pred_svm + 0.2 * y_pred_mlp
y_pred_final = (y_pred_ensemble > 0.4).astype(int)

print("最終的な予測結果:")
print(classification_report(y_test, y_pred_final, target_names=['human', 'bot']))


データの読み込みと前処理を開始します...
前処理が完了しました。
データの分割を行います...
テキストデータの変換を開始します...
テキストデータの変換が完了しました。
数値データの正規化を行います...
SMOTEを適用してデータのバランスを調整します...
データのバランス調整が完了しました。
モデルの学習を開始します...


  0%|                                                                                                         | 0/4 [00:00<?, ?it/s]

RandomForestの学習を開始します...


 25%|████████████████████████▎                                                                        | 1/4 [00:11<00:34, 11.55s/it]

RandomForestの学習が完了しました。学習時間: 11.55秒
GradientBoostingの学習を開始します...


...

 75%|████████████████████████████████████████████████████████████████████████                        | 3/4 [04:00<01:42, 102.22s/it]

SVMの学習が完了しました。学習時間: 193.52秒
MLPの学習を開始します...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [05:17<00:00, 79.41s/it]

MLPの学習が完了しました。学習時間: 77.30秒
アンサンブル予測を行います...


最終的な予測結果:
              precision    recall  f1-score   support

       human       0.91      0.94      0.93      1796
         bot       0.31      0.22      0.25       204

    accuracy                           0.87      2000
   macro avg       0.61      0.58      0.59      2000
weighted avg       0.85      0.87      0.86      2000



In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from scipy.sparse import hstack
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline
import re
from textblob import TextBlob
from tqdm import tqdm
import time
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

def extract_features(df):
    df['text_length'] = df['text'].str.len()
    df['has_url'] = df['text'].str.contains('http').astype(int)
    df['hour'] = pd.to_datetime(df['created_at']).dt.hour
    df['day_of_week'] = pd.to_datetime(df['created_at']).dt.dayofweek
    df['is_reply'] = df['in_reply_to_user_id'].notna().astype(int)
    df['hashtag_count'] = df['text'].apply(lambda x: len(re.findall(r'#\w+', x)))
    df['mention_count'] = df['text'].apply(lambda x: len(re.findall(r'@\w+', x)))
    df['retweet_like_ratio'] = df['retweet_count'] / (df['like_count'] + 1)
    df['sentiment'] = df['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
    df['subjectivity'] = df['text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    df['caps_ratio'] = df['text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)
    df['unique_words_ratio'] = df['text'].apply(lambda x: len(set(x.split())) / len(x.split()) if len(x.split()) > 0 else 0)
    df['tweet_frequency'] = df.groupby('author_id')['created_at'].transform('count')
    df['avg_word_length'] = df['text'].apply(lambda x: np.mean([len(word) for word in x.split()]) if len(x.split()) > 0 else 0)
    return df

print("データの読み込みと前処理を開始します...")
df = pd.read_csv('twibot_tweet0_new.csv')
df = extract_features(df)
print("前処理が完了しました。")

X_text = df['text']
numeric_features = ['retweet_count', 'like_count', 'text_length', 'has_url', 'hour', 'day_of_week', 'is_reply', 'hashtag_count', 'mention_count', 'retweet_like_ratio', 'sentiment', 'subjectivity', 'caps_ratio', 'unique_words_ratio', 'tweet_frequency', 'avg_word_length']
X_numeric = df[numeric_features]
y = df['label'].map({'human': 0, 'bot': 1})

X_train_text, X_test_text, X_train_numeric, X_test_numeric, y_train, y_test = train_test_split(X_text, X_numeric, y, test_size=0.2, random_state=42, stratify=y)

tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 3))
X_train_tfidf = tfidf.fit_transform(X_train_text)
X_test_tfidf = tfidf.transform(X_test_text)

scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.transform(X_test_numeric)

X_train_combined = hstack([X_train_tfidf, X_train_numeric_scaled])
X_test_combined = hstack([X_test_tfidf, X_test_numeric_scaled])

over = SMOTE(sampling_strategy=0.5, random_state=42)
under = RandomUnderSampler(sampling_strategy=0.8, random_state=42)
steps = [('over', over), ('under', under)]
pipeline = ImbPipeline(steps=steps)

X_train_resampled, y_train_resampled = pipeline.fit_resample(X_train_combined, y_train)

models = {
    'RandomForest': RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=200, random_state=42),
    'SVM': SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=42),
    'MLP': MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=300, random_state=42)
}

print("モデルの学習を開始します...")
for name, model in tqdm(models.items()):
    print(f"{name}の学習を開始します...")
    start_time = time.time()
    model.fit(X_train_resampled, y_train_resampled)
    end_time = time.time()
    print(f"{name}の学習が完了しました。学習時間: {end_time - start_time:.2f}秒")

print("アンサンブル予測を行います...")
y_pred_rf = models['RandomForest'].predict_proba(X_test_combined)[:, 1]
y_pred_gb = models['GradientBoosting'].predict_proba(X_test_combined)[:, 1]
y_pred_svm = models['SVM'].predict_proba(X_test_combined)[:, 1]
y_pred_mlp = models['MLP'].predict_proba(X_test_combined)[:, 1]

y_pred_ensemble = 0.3 * y_pred_rf + 0.3 * y_pred_gb + 0.2 * y_pred_svm + 0.2 * y_pred_mlp
y_pred_final = (y_pred_ensemble > 0.4).astype(int)

print("最終的な予測結果:")
print(classification_report(y_test, y_pred_final, target_names=['human', 'bot']))


データの読み込みと前処理を開始します...
前処理が完了しました。
モデルの学習を開始します...


  0%|                                                                                                         | 0/4 [00:00<?, ?it/s]

RandomForestの学習を開始します...


 25%|████████████████████████▎                                                                        | 1/4 [00:07<00:23,  7.82s/it]

RandomForestの学習が完了しました。学習時間: 7.82秒
GradientBoostingの学習を開始します...


...

 75%|████████████████████████████████████████████████████████████████████████▊                        | 3/4 [01:25<00:34, 34.77s/it]

SVMの学習が完了しました。学習時間: 59.43秒
MLPの学習を開始します...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:01<00:00, 30.46s/it]

MLPの学習が完了しました。学習時間: 36.18秒
アンサンブル予測を行います...


最終的な予測結果:
              precision    recall  f1-score   support

       human       0.92      0.92      0.92      1796
         bot       0.29      0.28      0.29       204

    accuracy                           0.86      2000
   macro avg       0.61      0.60      0.60      2000
weighted avg       0.85      0.86      0.86      2000



In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from scipy.sparse import hstack
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline
import re
from textblob import TextBlob
from tqdm import tqdm
import time
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

def extract_features(df):
    df['text_length'] = df['text'].str.len()
    df['has_url'] = df['text'].str.contains('http').astype(int)
    df['hour'] = pd.to_datetime(df['created_at']).dt.hour
    df['day_of_week'] = pd.to_datetime(df['created_at']).dt.dayofweek
    df['is_reply'] = df['in_reply_to_user_id'].notna().astype(int)
    df['hashtag_count'] = df['text'].apply(lambda x: len(re.findall(r'#\w+', x)))
    df['mention_count'] = df['text'].apply(lambda x: len(re.findall(r'@\w+', x)))
    df['retweet_like_ratio'] = df['retweet_count'] / (df['like_count'] + 1)
    df['sentiment'] = df['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
    df['subjectivity'] = df['text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    df['caps_ratio'] = df['text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)
    df['unique_words_ratio'] = df['text'].apply(lambda x: len(set(x.split())) / len(x.split()) if len(x.split()) > 0 else 0)
    df['tweet_frequency'] = df.groupby('author_id')['created_at'].transform('count')
    df['avg_word_length'] = df['text'].apply(lambda x: np.mean([len(word) for word in x.split()]) if len(x.split()) > 0 else 0)
    df['punctuation_count'] = df['text'].apply(lambda x: sum(1 for c in x if c in '.,;:!?'))
    return df

print("データの読み込みと前処理を開始します...")
df = pd.read_csv('twibot_tweet0_new.csv')
df = extract_features(df)
print("前処理が完了しました。")

X_text = df['text']
numeric_features = ['retweet_count', 'like_count', 'text_length', 'has_url', 'hour', 'day_of_week', 'is_reply', 'hashtag_count', 'mention_count', 'retweet_like_ratio', 'sentiment', 'subjectivity', 'caps_ratio', 'unique_words_ratio', 'tweet_frequency', 'avg_word_length', 'punctuation_count']
X_numeric = df[numeric_features]
y = df['label'].map({'human': 0, 'bot': 1})

X_train_text, X_test_text, X_train_numeric, X_test_numeric, y_train, y_test = train_test_split(X_text, X_numeric, y, test_size=0.2, random_state=42, stratify=y)

tfidf = TfidfVectorizer(max_features=6000, ngram_range=(1, 3))
X_train_tfidf = tfidf.fit_transform(X_train_text)
X_test_tfidf = tfidf.transform(X_test_text)

scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.transform(X_test_numeric)

X_train_combined = hstack([X_train_tfidf, X_train_numeric_scaled])
X_test_combined = hstack([X_test_tfidf, X_test_numeric_scaled])

over = SMOTE(sampling_strategy=0.5, random_state=42)
under = RandomUnderSampler(sampling_strategy=0.8, random_state=42)
steps = [('over', over), ('under', under)]
pipeline = ImbPipeline(steps=steps)

X_train_resampled, y_train_resampled = pipeline.fit_resample(X_train_combined, y_train)

models = {
    'RandomForest': RandomForestClassifier(n_estimators=300, class_weight='balanced', random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=300, learning_rate=0.05, random_state=42),
    'SVM': SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=42),
    'MLP': MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
}

print("モデルの学習を開始します...")
for name, model in tqdm(models.items()):
    print(f"{name}の学習を開始します...")
    start_time = time.time()
    model.fit(X_train_resampled, y_train_resampled)
    end_time = time.time()
    print(f"{name}の学習が完了しました。学習時間: {end_time - start_time:.2f}秒")

print("アンサンブル予測を行います...")
y_pred_rf = models['RandomForest'].predict_proba(X_test_combined)[:, 1]
y_pred_gb = models['GradientBoosting'].predict_proba(X_test_combined)[:, 1]
y_pred_svm = models['SVM'].predict_proba(X_test_combined)[:, 1]
y_pred_mlp = models['MLP'].predict_proba(X_test_combined)[:, 1]

y_pred_ensemble = 0.3 * y_pred_rf + 0.3 * y_pred_gb + 0.2 * y_pred_svm + 0.2 * y_pred_mlp
y_pred_final = (y_pred_ensemble > 0.35).astype(int)

print("最終的な予測結果:")
print(classification_report(y_test, y_pred_final, target_names=['human', 'bot']))


データの読み込みと前処理を開始します...
前処理が完了しました。
モデルの学習を開始します...


  0%|                                                                                                         | 0/4 [00:00<?, ?it/s]

RandomForestの学習を開始します...


 25%|████████████████████████▎                                                                        | 1/4 [00:12<00:37, 12.44s/it]

RandomForestの学習が完了しました。学習時間: 12.44秒
GradientBoostingの学習を開始します...


...

 75%|████████████████████████████████████████████████████████████████████████▊                        | 3/4 [01:41<00:39, 39.63s/it]

SVMの学習が完了しました。学習時間: 60.79秒
MLPの学習を開始します...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:45<00:00, 41.29s/it]

MLPの学習が完了しました。学習時間: 63.49秒
アンサンブル予測を行います...


最終的な予測結果:
              precision    recall  f1-score   support

       human       0.92      0.90      0.91      1796
         bot       0.25      0.31      0.28       204

    accuracy                           0.84      2000
   macro avg       0.59      0.60      0.59      2000
weighted avg       0.85      0.84      0.84      2000



In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from scipy.sparse import hstack
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline
import re
from textblob import TextBlob
from tqdm import tqdm
import time
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

def extract_features(df):
    df['text_length'] = df['text'].str.len()
    df['has_url'] = df['text'].str.contains('http').astype(int)
    df['hour'] = pd.to_datetime(df['created_at']).dt.hour
    df['day_of_week'] = pd.to_datetime(df['created_at']).dt.dayofweek
    df['is_reply'] = df['in_reply_to_user_id'].notna().astype(int)
    df['hashtag_count'] = df['text'].apply(lambda x: len(re.findall(r'#\w+', x)))
    df['mention_count'] = df['text'].apply(lambda x: len(re.findall(r'@\w+', x)))
    df['retweet_like_ratio'] = df['retweet_count'] / (df['like_count'] + 1)
    df['sentiment'] = df['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
    df['subjectivity'] = df['text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    df['caps_ratio'] = df['text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)
    df['unique_words_ratio'] = df['text'].apply(lambda x: len(set(x.split())) / len(x.split()) if len(x.split()) > 0 else 0)
    df['tweet_frequency'] = df.groupby('author_id')['created_at'].transform('count')
    df['avg_word_length'] = df['text'].apply(lambda x: np.mean([len(word) for word in x.split()]) if len(x.split()) > 0 else 0)
    df['punctuation_count'] = df['text'].apply(lambda x: sum(1 for c in x if c in '.,;:!?'))
    df['emoji_count'] = df['text'].apply(lambda x: len(re.findall(r'[\U0001F600-\U0001F64F]', x)))
    return df

print("データの読み込みと前処理を開始します...")
df = pd.read_csv('twibot_tweet0_new.csv')
df = extract_features(df)
print("前処理が完了しました。")

X_text = df['text']
numeric_features = ['retweet_count', 'like_count', 'text_length', 'has_url', 'hour', 'day_of_week', 'is_reply', 'hashtag_count', 'mention_count', 'retweet_like_ratio', 'sentiment', 'subjectivity', 'caps_ratio', 'unique_words_ratio', 'tweet_frequency', 'avg_word_length', 'punctuation_count', 'emoji_count']
X_numeric = df[numeric_features]
y = df['label'].map({'human': 0, 'bot': 1})

X_train_text, X_test_text, X_train_numeric, X_test_numeric, y_train, y_test = train_test_split(X_text, X_numeric, y, test_size=0.2, random_state=42, stratify=y)

tfidf = TfidfVectorizer(max_features=7000, ngram_range=(1, 3))
X_train_tfidf = tfidf.fit_transform(X_train_text)
X_test_tfidf = tfidf.transform(X_test_text)

scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.transform(X_test_numeric)

X_train_combined = hstack([X_train_tfidf, X_train_numeric_scaled])
X_test_combined = hstack([X_test_tfidf, X_test_numeric_scaled])

over = SMOTE(sampling_strategy=0.5, random_state=42)
under = RandomUnderSampler(sampling_strategy=0.8, random_state=42)
steps = [('over', over), ('under', under)]
pipeline = ImbPipeline(steps=steps)

X_train_resampled, y_train_resampled = pipeline.fit_resample(X_train_combined, y_train)

models = {
    'RandomForest': RandomForestClassifier(n_estimators=500, max_depth=20, class_weight='balanced', random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=500, learning_rate=0.05, max_depth=5, random_state=42),
    'SVM': SVC(kernel='rbf', C=10, gamma='scale', probability=True, class_weight='balanced', random_state=42),
    'MLP': MLPClassifier(hidden_layer_sizes=(100, 50, 25), max_iter=1000, alpha=0.0001, random_state=42)
}

print("モデルの学習を開始します...")
for name, model in tqdm(models.items()):
    print(f"{name}の学習を開始します...")
    start_time = time.time()
    model.fit(X_train_resampled, y_train_resampled)
    end_time = time.time()
    print(f"{name}の学習が完了しました。学習時間: {end_time - start_time:.2f}秒")

print("アンサンブル予測を行います...")
y_pred_rf = models['RandomForest'].predict_proba(X_test_combined)[:, 1]
y_pred_gb = models['GradientBoosting'].predict_proba(X_test_combined)[:, 1]
y_pred_svm = models['SVM'].predict_proba(X_test_combined)[:, 1]
y_pred_mlp = models['MLP'].predict_proba(X_test_combined)[:, 1]

y_pred_ensemble = 0.3 * y_pred_rf + 0.3 * y_pred_gb + 0.2 * y_pred_svm + 0.2 * y_pred_mlp
y_pred_final = (y_pred_ensemble > 0.3).astype(int)

print("最終的な予測結果:")
print(classification_report(y_test, y_pred_final, target_names=['human', 'bot']))


データの読み込みと前処理を開始します...
前処理が完了しました。
モデルの学習を開始します...


  0%|                                                                                                         | 0/4 [00:00<?, ?it/s]

RandomForestの学習を開始します...


 25%|████████████████████████▎                                                                        | 1/4 [00:07<00:21,  7.10s/it]

RandomForestの学習が完了しました。学習時間: 7.10秒
GradientBoostingの学習を開始します...


...

 75%|████████████████████████████████████████████████████████████████████████▊                        | 3/4 [02:43<01:02, 62.04s/it]

SVMの学習が完了しました。学習時間: 75.22秒
MLPの学習を開始します...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [03:26<00:00, 51.54s/it]

MLPの学習が完了しました。学習時間: 42.16秒
アンサンブル予測を行います...


最終的な予測結果:
              precision    recall  f1-score   support

       human       0.92      0.88      0.90      1796
         bot       0.26      0.36      0.30       204

    accuracy                           0.83      2000
   macro avg       0.59      0.62      0.60      2000
weighted avg       0.86      0.83      0.84      2000



In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from scipy.sparse import hstack
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.pipeline import Pipeline as ImbPipeline
import re
from textblob import TextBlob
from tqdm import tqdm
import time
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif

def extract_features(df):
    df['text_length'] = df['text'].str.len()
    df['has_url'] = df['text'].str.contains('http').astype(int)
    df['hour'] = pd.to_datetime(df['created_at']).dt.hour
    df['day_of_week'] = pd.to_datetime(df['created_at']).dt.dayofweek
    df['is_reply'] = df['in_reply_to_user_id'].notna().astype(int)
    df['hashtag_count'] = df['text'].apply(lambda x: len(re.findall(r'#\w+', x)))
    df['mention_count'] = df['text'].apply(lambda x: len(re.findall(r'@\w+', x)))
    df['retweet_like_ratio'] = df['retweet_count'] / (df['like_count'] + 1)
    df['sentiment'] = df['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
    df['subjectivity'] = df['text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    df['caps_ratio'] = df['text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)
    df['unique_words_ratio'] = df['text'].apply(lambda x: len(set(x.split())) / len(x.split()) if len(x.split()) > 0 else 0)
    df['tweet_frequency'] = df.groupby('author_id')['created_at'].transform('count')
    df['avg_word_length'] = df['text'].apply(lambda x: np.mean([len(word) for word in x.split()]) if len(x.split()) > 0 else 0)
    df['punctuation_count'] = df['text'].apply(lambda x: sum(1 for c in x if c in '.,;:!?'))
    df['emoji_count'] = df['text'].apply(lambda x: len(re.findall(r'[\U0001F600-\U0001F64F]', x)))
    df['exclamation_count'] = df['text'].apply(lambda x: x.count('!'))
    df['question_count'] = df['text'].apply(lambda x: x.count('?'))
    return df

print("データの読み込みと前処理を開始します...")
df = pd.read_csv('twibot_tweet0_new.csv')
df = extract_features(df)
print("前処理が完了しました。")

X_text = df['text']
numeric_features = ['retweet_count', 'like_count', 'text_length', 'has_url', 'hour', 'day_of_week', 'is_reply', 'hashtag_count', 'mention_count', 'retweet_like_ratio', 'sentiment', 'subjectivity', 'caps_ratio', 'unique_words_ratio', 'tweet_frequency', 'avg_word_length', 'punctuation_count', 'emoji_count', 'exclamation_count', 'question_count']
X_numeric = df[numeric_features]
y = df['label'].map({'human': 0, 'bot': 1})

X_train_text, X_test_text, X_train_numeric, X_test_numeric, y_train, y_test = train_test_split(X_text, X_numeric, y, test_size=0.2, random_state=42, stratify=y)

tfidf = TfidfVectorizer(max_features=8000, ngram_range=(1, 3))
X_train_tfidf = tfidf.fit_transform(X_train_text)
X_test_tfidf = tfidf.transform(X_test_text)

scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.transform(X_test_numeric)

X_train_combined = hstack([X_train_tfidf, X_train_numeric_scaled])
X_test_combined = hstack([X_test_tfidf, X_test_numeric_scaled])

selector = SelectKBest(f_classif, k=5000)
X_train_selected = selector.fit_transform(X_train_combined, y_train)
X_test_selected = selector.transform(X_test_combined)

over = SMOTE(sampling_strategy=0.5, random_state=42)
under = TomekLinks(sampling_strategy='majority')
steps = [('over', over), ('under', under)]
pipeline = ImbPipeline(steps=steps)

X_train_resampled, y_train_resampled = pipeline.fit_resample(X_train_selected, y_train)

models = {
    'RandomForest': RandomForestClassifier(n_estimators=500, max_depth=20, class_weight='balanced', random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=500, learning_rate=0.05, max_depth=5, random_state=42),
    'SVM': SVC(kernel='rbf', C=10, gamma='scale', probability=True, class_weight='balanced', random_state=42),
    'MLP': MLPClassifier(hidden_layer_sizes=(100, 50, 25), max_iter=1000, alpha=0.0001, random_state=42)
}

print("モデルの学習を開始します...")
for name, model in tqdm(models.items()):
    print(f"{name}の学習を開始します...")
    start_time = time.time()
    model.fit(X_train_resampled, y_train_resampled)
    end_time = time.time()
    print(f"{name}の学習が完了しました。学習時間: {end_time - start_time:.2f}秒")

print("アンサンブル予測を行います...")
y_pred_rf = models['RandomForest'].predict_proba(X_test_selected)[:, 1]
y_pred_gb = models['GradientBoosting'].predict_proba(X_test_selected)[:, 1]
y_pred_svm = models['SVM'].predict_proba(X_test_selected)[:, 1]
y_pred_mlp = models['MLP'].predict_proba(X_test_selected)[:, 1]

y_pred_ensemble = 0.3 * y_pred_rf + 0.3 * y_pred_gb + 0.2 * y_pred_svm + 0.2 * y_pred_mlp
y_pred_final = (y_pred_ensemble > 0.3).astype(int)

print("最終的な予測結果:")
print(classification_report(y_test, y_pred_final, target_names=['human', 'bot']))


データの読み込みと前処理を開始します...
前処理が完了しました。


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [0 0 0 0] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


モデルの学習を開始します...


  0%|                                                                                                         | 0/4 [00:00<?, ?it/s]

RandomForestの学習を開始します...


...

 75%|████████████████████████████████████████████████████████████████████████▊                        | 3/4 [02:07<00:47, 47.98s/it]

SVMの学習が完了しました。学習時間: 56.71秒
MLPの学習を開始します...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:48<00:00, 42.18s/it]

MLPの学習が完了しました。学習時間: 41.05秒
アンサンブル予測を行います...


最終的な予測結果:
              precision    recall  f1-score   support

       human       0.92      0.94      0.93      1796
         bot       0.33      0.26      0.29       204

    accuracy                           0.87      2000
   macro avg       0.62      0.60      0.61      2000
weighted avg       0.86      0.87      0.86      2000



In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from scipy.sparse import hstack
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.pipeline import Pipeline as ImbPipeline
import re
from textblob import TextBlob
from tqdm import tqdm
import time
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectKBest, chi2, f_classif

def extract_features(df):
    df['text_length'] = df['text'].str.len()
    df['has_url'] = df['text'].str.contains('http').astype(int)
    df['hour'] = pd.to_datetime(df['created_at']).dt.hour
    df['day_of_week'] = pd.to_datetime(df['created_at']).dt.dayofweek
    df['is_reply'] = df['in_reply_to_user_id'].notna().astype(int)
    df['hashtag_count'] = df['text'].apply(lambda x: len(re.findall(r'#\w+', x)))
    df['mention_count'] = df['text'].apply(lambda x: len(re.findall(r'@\w+', x)))
    df['retweet_like_ratio'] = df['retweet_count'] / (df['like_count'] + 1)
    df['sentiment'] = df['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
    df['subjectivity'] = df['text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    df['caps_ratio'] = df['text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)
    df['unique_words_ratio'] = df['text'].apply(lambda x: len(set(x.split())) / len(x.split()) if len(x.split()) > 0 else 0)
    df['tweet_frequency'] = df.groupby('author_id')['created_at'].transform('count')
    df['avg_word_length'] = df['text'].apply(lambda x: np.mean([len(word) for word in x.split()]) if len(x.split()) > 0 else 0)
    df['punctuation_count'] = df['text'].apply(lambda x: sum(1 for c in x if c in '.,;:!?'))
    df['emoji_count'] = df['text'].apply(lambda x: len(re.findall(r'[\U0001F600-\U0001F64F]', x)))
    df['exclamation_count'] = df['text'].apply(lambda x: x.count('!'))
    df['question_count'] = df['text'].apply(lambda x: x.count('?'))
    df['uppercase_word_count'] = df['text'].apply(lambda x: sum(1 for word in x.split() if word.isupper()))
    return df

print("データの読み込みと前処理を開始します...")
df = pd.read_csv('twibot_tweet0_new.csv')
df = extract_features(df)
print("前処理が完了しました。")

X_text = df['text']
numeric_features = ['retweet_count', 'like_count', 'text_length', 'has_url', 'hour', 'day_of_week', 'is_reply', 'hashtag_count', 'mention_count', 'retweet_like_ratio', 'sentiment', 'subjectivity', 'caps_ratio', 'unique_words_ratio', 'tweet_frequency', 'avg_word_length', 'punctuation_count', 'emoji_count', 'exclamation_count', 'question_count', 'uppercase_word_count']
X_numeric = df[numeric_features]
y = df['label'].map({'human': 0, 'bot': 1})

X_train_text, X_test_text, X_train_numeric, X_test_numeric, y_train, y_test = train_test_split(X_text, X_numeric, y, test_size=0.2, random_state=42, stratify=y)

tfidf = TfidfVectorizer(max_features=10000, ngram_range=(1, 3))
X_train_tfidf = tfidf.fit_transform(X_train_text)
X_test_tfidf = tfidf.transform(X_test_text)

scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.transform(X_test_numeric)

# テキストデータの特徴量選択
selector_text = SelectKBest(chi2, k=5000)
X_train_text_selected = selector_text.fit_transform(X_train_tfidf, y_train)
X_test_text_selected = selector_text.transform(X_test_tfidf)

# 数値データの特徴量選択
selector_numeric = SelectKBest(f_classif, k=20)
X_train_numeric_selected = selector_numeric.fit_transform(X_train_numeric_scaled, y_train)
X_test_numeric_selected = selector_numeric.transform(X_test_numeric_scaled)

# 選択された特徴量の結合
X_train_selected = hstack([X_train_text_selected, X_train_numeric_selected])
X_test_selected = hstack([X_test_text_selected, X_test_numeric_selected])

over = SMOTE(sampling_strategy=0.5, random_state=42)
under = TomekLinks(sampling_strategy='majority')
steps = [('over', over), ('under', under)]
pipeline = ImbPipeline(steps=steps)

X_train_resampled, y_train_resampled = pipeline.fit_resample(X_train_selected, y_train)

models = {
    'RandomForest': RandomForestClassifier(n_estimators=500, max_depth=20, class_weight='balanced', random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=500, learning_rate=0.05, max_depth=5, random_state=42),
    'SVM': SVC(kernel='rbf', C=10, gamma='scale', probability=True, class_weight='balanced', random_state=42),
    'MLP': MLPClassifier(hidden_layer_sizes=(100, 50, 25), max_iter=1000, alpha=0.0001, random_state=42)
}

print("モデルの学習を開始します...")
for name, model in tqdm(models.items()):
    print(f"{name}の学習を開始します...")
    start_time = time.time()
    model.fit(X_train_resampled, y_train_resampled)
    end_time = time.time()
    print(f"{name}の学習が完了しました。学習時間: {end_time - start_time:.2f}秒")

print("アンサンブル予測を行います...")
y_pred_rf = models['RandomForest'].predict_proba(X_test_selected)[:, 1]
y_pred_gb = models['GradientBoosting'].predict_proba(X_test_selected)[:, 1]
y_pred_svm = models['SVM'].predict_proba(X_test_selected)[:, 1]
y_pred_mlp = models['MLP'].predict_proba(X_test_selected)[:, 1]

y_pred_ensemble = 0.3 * y_pred_rf + 0.3 * y_pred_gb + 0.2 * y_pred_svm + 0.2 * y_pred_mlp
y_pred_final = (y_pred_ensemble > 0.3).astype(int)

print("最終的な予測結果:")
print(classification_report(y_test, y_pred_final, target_names=['human', 'bot']))


データの読み込みと前処理を開始します...
前処理が完了しました。


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 3  7  8 17] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


モデルの学習を開始します...


  0%|                                                                                                         | 0/4 [00:00<?, ?it/s]

RandomForestの学習を開始します...


...

 75%|████████████████████████████████████████████████████████████████████████▊                        | 3/4 [02:10<00:48, 48.68s/it]

SVMの学習が完了しました。学習時間: 56.06秒
MLPの学習を開始します...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:58<00:00, 44.50s/it]

MLPの学習が完了しました。学習時間: 47.80秒
アンサンブル予測を行います...


最終的な予測結果:
              precision    recall  f1-score   support

       human       0.92      0.93      0.93      1796
         bot       0.33      0.28      0.30       204

    accuracy                           0.87      2000
   macro avg       0.62      0.61      0.61      2000
weighted avg       0.86      0.87      0.86      2000



In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from scipy.sparse import hstack
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.pipeline import Pipeline as ImbPipeline
import re
from textblob import TextBlob
from tqdm import tqdm
import time
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import StratifiedKFold

def extract_features(df):
    df['text_length'] = df['text'].str.len()
    df['has_url'] = df['text'].str.contains('http').astype(int)
    df['hour'] = pd.to_datetime(df['created_at']).dt.hour
    df['day_of_week'] = pd.to_datetime(df['created_at']).dt.dayofweek
    df['is_reply'] = df['in_reply_to_user_id'].notna().astype(int)
    df['hashtag_count'] = df['text'].apply(lambda x: len(re.findall(r'#\w+', x)))
    df['mention_count'] = df['text'].apply(lambda x: len(re.findall(r'@\w+', x)))
    df['retweet_like_ratio'] = df['retweet_count'] / (df['like_count'] + 1)
    df['sentiment'] = df['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
    df['subjectivity'] = df['text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    df['caps_ratio'] = df['text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)
    df['unique_words_ratio'] = df['text'].apply(lambda x: len(set(x.split())) / len(x.split()) if len(x.split()) > 0 else 0)
    df['tweet_frequency'] = df.groupby('author_id')['created_at'].transform('count')
    df['avg_word_length'] = df['text'].apply(lambda x: np.mean([len(word) for word in x.split()]) if len(x.split()) > 0 else 0)
    df['punctuation_count'] = df['text'].apply(lambda x: sum(1 for c in x if c in '.,;:!?'))
    df['emoji_count'] = df['text'].apply(lambda x: len(re.findall(r'[\U0001F600-\U0001F64F]', x)))
    df['exclamation_count'] = df['text'].apply(lambda x: x.count('!'))
    df['question_count'] = df['text'].apply(lambda x: x.count('?'))
    df['uppercase_word_count'] = df['text'].apply(lambda x: sum(1 for word in x.split() if word.isupper()))
    return df

print("データの読み込みと前処理を開始します...")
df = pd.read_csv('twibot_tweet0_new.csv')
df = extract_features(df)
print("前処理が完了しました。")

X_text = df['text']
numeric_features = ['retweet_count', 'like_count', 'text_length', 'has_url', 'hour', 'day_of_week', 'is_reply', 'hashtag_count', 'mention_count', 'retweet_like_ratio', 'sentiment', 'subjectivity', 'caps_ratio', 'unique_words_ratio', 'tweet_frequency', 'avg_word_length', 'punctuation_count', 'emoji_count', 'exclamation_count', 'question_count', 'uppercase_word_count']
X_numeric = df[numeric_features]
y = df['label'].map({'human': 0, 'bot': 1})

X_train_text, X_test_text, X_train_numeric, X_test_numeric, y_train, y_test = train_test_split(X_text, X_numeric, y, test_size=0.2, random_state=42, stratify=y)

tfidf = TfidfVectorizer(max_features=15000, ngram_range=(1, 3))
X_train_tfidf = tfidf.fit_transform(X_train_text)
X_test_tfidf = tfidf.transform(X_test_text)

scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.transform(X_test_numeric)

selector_text = SelectKBest(chi2, k=7000)
X_train_text_selected = selector_text.fit_transform(X_train_tfidf, y_train)
X_test_text_selected = selector_text.transform(X_test_tfidf)

selector_numeric = SelectKBest(f_classif, k=15)
X_train_numeric_selected = selector_numeric.fit_transform(X_train_numeric_scaled, y_train)
X_test_numeric_selected = selector_numeric.transform(X_test_numeric_scaled)

X_train_selected = hstack([X_train_text_selected, X_train_numeric_selected])
X_test_selected = hstack([X_test_text_selected, X_test_numeric_selected])

over = SMOTE(sampling_strategy=0.5, random_state=42)
under = TomekLinks(sampling_strategy='majority')
steps = [('over', over), ('under', under)]
pipeline = ImbPipeline(steps=steps)

X_train_resampled, y_train_resampled = pipeline.fit_resample(X_train_selected, y_train)

models = {
    'RandomForest': RandomForestClassifier(n_estimators=500, max_depth=20, class_weight='balanced', random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=500, learning_rate=0.05, max_depth=5, random_state=42),
    'SVM': SVC(kernel='rbf', C=10, gamma='scale', probability=True, class_weight='balanced', random_state=42),
    'MLP': MLPClassifier(hidden_layer_sizes=(100, 50, 25), max_iter=1000, alpha=0.0001, random_state=42)
}

print("モデルの学習を開始します...")
for name, model in tqdm(models.items()):
    print(f"{name}の学習を開始します...")
    start_time = time.time()
    model.fit(X_train_resampled, y_train_resampled)
    end_time = time.time()
    print(f"{name}の学習が完了しました。学習時間: {end_time - start_time:.2f}秒")

print("アンサンブル予測を行います...")
y_pred_rf = models['RandomForest'].predict_proba(X_test_selected)[:, 1]
y_pred_gb = models['GradientBoosting'].predict_proba(X_test_selected)[:, 1]
y_pred_svm = models['SVM'].predict_proba(X_test_selected)[:, 1]
y_pred_mlp = models['MLP'].predict_proba(X_test_selected)[:, 1]

y_pred_ensemble = 0.3 * y_pred_rf + 0.3 * y_pred_gb + 0.2 * y_pred_svm + 0.2 * y_pred_mlp
y_pred_final = (y_pred_ensemble > 0.3).astype(int)

print("最終的な予測結果:")
print(classification_report(y_test, y_pred_final, target_names=['human', 'bot']))


データの読み込みと前処理を開始します...
前処理が完了しました。


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 3  7  8 17] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


モデルの学習を開始します...


  0%|                                                                                                         | 0/4 [00:00<?, ?it/s]

RandomForestの学習を開始します...


...

 75%|████████████████████████████████████████████████████████████████████████▊                        | 3/4 [02:15<00:51, 51.14s/it]

SVMの学習が完了しました。学習時間: 61.36秒
MLPの学習を開始します...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [03:34<00:00, 53.72s/it]

MLPの学習が完了しました。学習時間: 79.40秒
アンサンブル予測を行います...


最終的な予測結果:
              precision    recall  f1-score   support

       human       0.92      0.95      0.93      1796
         bot       0.36      0.27      0.31       204

    accuracy                           0.88      2000
   macro avg       0.64      0.61      0.62      2000
weighted avg       0.86      0.88      0.87      2000



In [58]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from scipy.sparse import hstack
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.pipeline import Pipeline as ImbPipeline
import re
from textblob import TextBlob
from tqdm import tqdm
import time
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold
from sklearn.model_selection import StratifiedKFold
from transformers import BertTokenizer, BertModel
import torch
import xgboost as xgb

def extract_features(df):
    df['text_length'] = df['text'].str.len()
    df['has_url'] = df['text'].str.contains('http').astype(int)
    df['hour'] = pd.to_datetime(df['created_at']).dt.hour
    df['day_of_week'] = pd.to_datetime(df['created_at']).dt.dayofweek
    df['is_reply'] = df['in_reply_to_user_id'].notna().astype(int)
    df['hashtag_count'] = df['text'].apply(lambda x: len(re.findall(r'#\w+', x)))
    df['mention_count'] = df['text'].apply(lambda x: len(re.findall(r'@\w+', x)))
    df['retweet_like_ratio'] = df['retweet_count'] / (df['like_count'] + 1)
    df['sentiment'] = df['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
    df['subjectivity'] = df['text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    df['caps_ratio'] = df['text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)
    df['unique_words_ratio'] = df['text'].apply(lambda x: len(set(x.split())) / len(x.split()) if len(x.split()) > 0 else 0)
    df['tweet_frequency'] = df.groupby('author_id')['created_at'].transform('count')
    df['avg_word_length'] = df['text'].apply(lambda x: np.mean([len(word) for word in x.split()]) if len(x.split()) > 0 else 0)
    df['punctuation_count'] = df['text'].apply(lambda x: sum(1 for c in x if c in '.,;:!?'))
    df['emoji_count'] = df['text'].apply(lambda x: len(re.findall(r'[\U0001F600-\U0001F64F]', x)))
    df['exclamation_count'] = df['text'].apply(lambda x: x.count('!'))
    df['question_count'] = df['text'].apply(lambda x: x.count('?'))
    df['uppercase_word_count'] = df['text'].apply(lambda x: sum(1 for word in x.split() if word.isupper()))
    return df

def get_bert_embeddings(texts, model, tokenizer):
    embeddings = []
    for text in tqdm(texts):
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
    return np.array(embeddings)

class XGBoostProgressCallback(xgb.callback.TrainingCallback):
    def after_iteration(self, model, epoch, evals_log):
        if (epoch + 1) % 50 == 0 or (epoch + 1) == model.num_boosted_rounds():
            print(f'XGBoost iteration {epoch + 1}/{model.num_boosted_rounds()}')
        return False

print("データの読み込みと前処理を開始します...")
df = pd.read_csv('twibot_tweet0_new.csv')
df = extract_features(df)
print("前処理が完了しました。")

X_text = df['text']
numeric_features = ['retweet_count', 'like_count', 'text_length', 'has_url', 'hour', 'day_of_week', 'is_reply', 'hashtag_count', 'mention_count', 'retweet_like_ratio', 'sentiment', 'subjectivity', 'caps_ratio', 'unique_words_ratio', 'tweet_frequency', 'avg_word_length', 'punctuation_count', 'emoji_count', 'exclamation_count', 'question_count', 'uppercase_word_count']
X_numeric = df[numeric_features]
y = df['label'].map({'human': 0, 'bot': 1})

X_train_text, X_test_text, X_train_numeric, X_test_numeric, y_train, y_test = train_test_split(X_text, X_numeric, y, test_size=0.2, random_state=42, stratify=y)

tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1, 3))
X_train_tfidf = tfidf.fit_transform(X_train_text)
X_test_tfidf = tfidf.transform(X_test_text)

scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.transform(X_test_numeric)

print("BERTモデルを使用して文章の埋め込みを生成します...")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
X_train_bert = get_bert_embeddings(X_train_text, model, tokenizer)
X_test_bert = get_bert_embeddings(X_test_text, model, tokenizer)

X_train_combined = np.hstack([X_train_tfidf.toarray(), X_train_numeric_scaled, X_train_bert])
X_test_combined = np.hstack([X_test_tfidf.toarray(), X_test_numeric_scaled, X_test_bert])

# 分散の低い特徴量を除去
variance_threshold = VarianceThreshold(threshold=0.01)
X_train_var_selected = variance_threshold.fit_transform(X_train_combined)
X_test_var_selected = variance_threshold.transform(X_test_combined)

# f_classifを使用して特徴量を選択
selector = SelectKBest(f_classif, k="all")
X_train_selected = selector.fit_transform(X_train_var_selected, y_train)
X_test_selected = selector.transform(X_test_var_selected)

over = SMOTE(sampling_strategy=0.5, random_state=42)
under = TomekLinks(sampling_strategy='majority')
steps = [('over', over), ('under', under)]
pipeline = ImbPipeline(steps=steps)

X_train_resampled, y_train_resampled = pipeline.fit_resample(X_train_selected, y_train)

models = {
    'RandomForest': RandomForestClassifier(n_estimators=500, max_depth=20, class_weight='balanced', random_state=42),
    'XGBoost': xgb.XGBClassifier(n_estimators=500, learning_rate=0.05, max_depth=5, random_state=42, callbacks=[XGBoostProgressCallback()]),
    'SVM': SVC(kernel='rbf', C=10, gamma='scale', probability=True, class_weight='balanced', random_state=42),
    'MLP': MLPClassifier(hidden_layer_sizes=(100, 50, 25), max_iter=1000, alpha=0.0001, random_state=42)
}

print("モデルの学習を開始します...")
for name, model in tqdm(models.items()):
    print(f"{name}の学習を開始します...")
    start_time = time.time()
    model.fit(X_train_resampled, y_train_resampled)
    end_time = time.time()
    print(f"{name}の学習が完了しました。学習時間: {end_time - start_time:.2f}秒")

print("アンサンブル予測を行います...")
y_pred_rf = models['RandomForest'].predict_proba(X_test_selected)[:, 1]
y_pred_xgb = models['XGBoost'].predict_proba(X_test_selected)[:, 1]
y_pred_svm = models['SVM'].predict_proba(X_test_selected)[:, 1]
y_pred_mlp = models['MLP'].predict_proba(X_test_selected)[:, 1]

y_pred_ensemble = 0.3 * y_pred_rf + 0.3 * y_pred_xgb + 0.2 * y_pred_svm + 0.2 * y_pred_mlp
y_pred_final = (y_pred_ensemble > 0.3).astype(int)

print("最終的な予測結果:")
print(classification_report(y_test, y_pred_final, target_names=['human', 'bot']))


データの読み込みと前処理を開始します...
前処理が完了しました。
BERTモデルを使用して文章の埋め込みを生成します...


100%|███████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [02:13<00:00, 15.01it/s]


モデルの学習を開始します...


  0%|                                                                                                         | 0/4 [00:00<?, ?it/s]

RandomForestの学習を開始します...


...

 75%|████████████████████████████████████████████████████████████████████████                        | 3/4 [06:50<02:24, 144.51s/it]

SVMの学習が完了しました。学習時間: 214.47秒
MLPの学習を開始します...


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [07:03<00:00, 105.94s/it]

MLPの学習が完了しました。学習時間: 13.17秒
アンサンブル予測を行います...


最終的な予測結果:
              precision    recall  f1-score   support

       human       0.92      0.96      0.94      1796
         bot       0.40      0.25      0.31       204

    accuracy                           0.89      2000
   macro avg       0.66      0.60      0.62      2000
weighted avg       0.87      0.89      0.87      2000



In [60]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from scipy.sparse import hstack
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import TomekLinks
from imblearn.pipeline import Pipeline as ImbPipeline
import re
from textblob import TextBlob
from tqdm import tqdm
import time
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold
from sklearn.model_selection import StratifiedKFold
from transformers import BertTokenizer, BertModel
import torch
import xgboost as xgb
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import RandomizedSearchCV

def extract_features(df):
    df['text_length'] = df['text'].str.len()
    df['has_url'] = df['text'].str.contains('http').astype(int)
    df['hour'] = pd.to_datetime(df['created_at']).dt.hour
    df['day_of_week'] = pd.to_datetime(df['created_at']).dt.dayofweek
    df['is_reply'] = df['in_reply_to_user_id'].notna().astype(int)
    df['hashtag_count'] = df['text'].apply(lambda x: len(re.findall(r'#\w+', x)))
    df['mention_count'] = df['text'].apply(lambda x: len(re.findall(r'@\w+', x)))
    df['retweet_like_ratio'] = df['retweet_count'] / (df['like_count'] + 1)
    df['sentiment'] = df['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
    df['subjectivity'] = df['text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    df['caps_ratio'] = df['text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)
    df['unique_words_ratio'] = df['text'].apply(lambda x: len(set(x.split())) / len(x.split()) if len(x.split()) > 0 else 0)
    df['tweet_frequency'] = df.groupby('author_id')['created_at'].transform('count')
    df['avg_word_length'] = df['text'].apply(lambda x: np.mean([len(word) for word in x.split()]) if len(x.split()) > 0 else 0)
    df['punctuation_count'] = df['text'].apply(lambda x: sum(1 for c in x if c in '.,;:!?'))
    df['emoji_count'] = df['text'].apply(lambda x: len(re.findall(r'[\U0001F600-\U0001F64F]', x)))
    df['exclamation_count'] = df['text'].apply(lambda x: x.count('!'))
    df['question_count'] = df['text'].apply(lambda x: x.count('?'))
    df['uppercase_word_count'] = df['text'].apply(lambda x: sum(1 for word in x.split() if word.isupper()))
    df['url_count'] = df['text'].apply(lambda x: len(re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', x)))
    df['digit_count'] = df['text'].apply(lambda x: sum(c.isdigit() for c in x))
    df['special_char_count'] = df['text'].apply(lambda x: len(re.findall(r'[^a-zA-Z0-9\s]', x)))
    return df

def get_bert_embeddings(texts, model, tokenizer):
    embeddings = []
    for text in tqdm(texts):
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
    return np.array(embeddings)

class XGBoostProgressCallback(xgb.callback.TrainingCallback):
    def after_iteration(self, model, epoch, evals_log):
        if (epoch + 1) % 50 == 0 or (epoch + 1) == model.num_boosted_rounds():
            print(f'XGBoost iteration {epoch + 1}/{model.num_boosted_rounds()}')
        return False

print("データの読み込みと前処理を開始します...")
df = pd.read_csv('twibot_tweet0_new.csv')
df = extract_features(df)
print("前処理が完了しました。")

X_text = df['text']
numeric_features = ['retweet_count', 'like_count', 'text_length', 'has_url', 'hour', 'day_of_week', 'is_reply', 'hashtag_count', 'mention_count', 'retweet_like_ratio', 'sentiment', 'subjectivity', 'caps_ratio', 'unique_words_ratio', 'tweet_frequency', 'avg_word_length', 'punctuation_count', 'emoji_count', 'exclamation_count', 'question_count', 'uppercase_word_count', 'url_count', 'digit_count', 'special_char_count']
X_numeric = df[numeric_features]
y = df['label'].map({'human': 0, 'bot': 1})

X_train_text, X_test_text, X_train_numeric, X_test_numeric, y_train, y_test = train_test_split(X_text, X_numeric, y, test_size=0.2, random_state=42, stratify=y)

tfidf = TfidfVectorizer(max_features=25000, ngram_range=(1, 3))
X_train_tfidf = tfidf.fit_transform(X_train_text)
X_test_tfidf = tfidf.transform(X_test_text)

scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.transform(X_test_numeric)

print("BERTモデルを使用して文章の埋め込みを生成します...")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
X_train_bert = get_bert_embeddings(X_train_text, model, tokenizer)
X_test_bert = get_bert_embeddings(X_test_text, model, tokenizer)

X_train_combined = np.hstack([X_train_tfidf.toarray(), X_train_numeric_scaled, X_train_bert])
X_test_combined = np.hstack([X_test_tfidf.toarray(), X_test_numeric_scaled, X_test_bert])

# 分散の低い特徴量を除去
variance_threshold = VarianceThreshold(threshold=0.005)
X_train_var_selected = variance_threshold.fit_transform(X_train_combined)
X_test_var_selected = variance_threshold.transform(X_test_combined)

# f_classifを使用して特徴量を選択
selector = SelectKBest(f_classif, k="all")
X_train_selected = selector.fit_transform(X_train_var_selected, y_train)
X_test_selected = selector.transform(X_test_var_selected)

over = ADASYN(sampling_strategy=0.7, random_state=42)
under = TomekLinks(sampling_strategy='majority')
steps = [('over', over), ('under', under)]
pipeline = ImbPipeline(steps=steps)

X_train_resampled, y_train_resampled = pipeline.fit_resample(X_train_selected, y_train)

rf = RandomForestClassifier(n_estimators=1000, max_depth=30, class_weight='balanced', random_state=42, n_jobs=-1)
xgb_model = xgb.XGBClassifier(n_estimators=1000, learning_rate=0.05, max_depth=7, random_state=42, callbacks=[XGBoostProgressCallback()], n_jobs=-1)
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, class_weight='balanced', random_state=42)
mlp = MLPClassifier(hidden_layer_sizes=(200, 100, 50), max_iter=1000, alpha=0.0001, random_state=42)

voting_clf = VotingClassifier(
    estimators=[('rf', rf), ('xgb', xgb_model), ('svm', svm), ('mlp', mlp)],
    voting='soft',
    weights=[0.3, 0.3, 0.2, 0.2]
)

param_dist = {
    'rf__n_estimators': [800, 1000, 1200],
    'rf__max_depth': [25, 30, 35],
    'xgb__n_estimators': [800, 1000, 1200],
    'xgb__learning_rate': [0.01, 0.05, 0.1],
    'xgb__max_depth': [5, 7, 9],
    'svm__C': [1, 10, 100],
    'svm__gamma': ['scale', 'auto'],
    'mlp__hidden_layer_sizes': [(100, 50), (200, 100), (200, 100, 50)],
    'mlp__alpha': [0.0001, 0.001, 0.01],
    'voting': ['soft'],
    'weights': [[0.3, 0.3, 0.2, 0.2], [0.25, 0.25, 0.25, 0.25], [0.4, 0.4, 0.1, 0.1]]
}

print("ランダムサーチによるハイパーパラメータ最適化を開始します...")
random_search = RandomizedSearchCV(
    voting_clf,
    param_distributions=param_dist,
    n_iter=20,
    cv=3,
    scoring=make_scorer(f1_score, average='weighted'),
    n_jobs=-1,
    random_state=42,
    verbose=2
)

random_search.fit(X_train_resampled, y_train_resampled)

print("最適なパラメータ:", random_search.best_params_)
print("最高スコア:", random_search.best_score_)

best_model = random_search.best_estimator_

print("テストデータで最終評価を行います...")
y_pred = best_model.predict(X_test_selected)

print("最終的な予測結果:")
print(classification_report(y_test, y_pred, target_names=['human', 'bot']))


データの読み込みと前処理を開始します...
前処理が完了しました。
BERTモデルを使用して文章の埋め込みを生成します...


100%|███████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [01:41<00:00, 19.78it/s]


ランダムサーチによるハイパーパラメータ最適化を開始します...
Fitting 3 folds for each of 20 candidates, totalling 60 fits
XGBoost iteration 1/1
XGBoost iteration 2/2
XGBoost iteration 3/3
XGBoost iteration 4/4
XGBoost iteration 5/5
XGBoost iteration 6/6
XGBoost iteration 7/7
XGBoost iteration 8/8
XGBoost iteration 9/9
XGBoost iteration 10/10
XGBoost iteration 11/11
XGBoost iteration 12/12
XGBoost iteration 13/13
XGBoost iteration 14/14
XGBoost iteration 15/15
XGBoost iteration 16/16
XGBoost iteration 17/17
XGBoost iteration 18/18
XGBoost iteration 19/19
XGBoost iteration 20/20
XGBoost iteration 21/21
XGBoost iteration 22/22
XGBoost iteration 23/23
XGBoost iteration 24/24
XGBoost iteration 25/25
XGBoost iteration 26/26
XGBoost iteration 27/27
XGBoost iteration 28/28
XGBoost iteration 29/29
XGBoost iteration 30/30
XGBoost iteration 31/31
XGBoost iteration 32/32
XGBoost iteration 33/33
XGBoost iteration 34/34
XGBoost iteration 35/35
XGBoost iteration 36/36
XGBoost iteration 37/37
XGBoost iteration 38/38
XGBoost itera

In [64]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from scipy.sparse import hstack
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.pipeline import Pipeline as ImbPipeline
import re
from textblob import TextBlob
from tqdm import tqdm
import time
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold
from sklearn.model_selection import StratifiedKFold
from transformers import BertTokenizer, BertModel
import torch
import xgboost as xgb

def extract_features(df):
    df['text_length'] = df['text'].str.len()
    df['has_url'] = df['text'].str.contains('http').astype(int)
    df['hour'] = pd.to_datetime(df['created_at']).dt.hour
    df['day_of_week'] = pd.to_datetime(df['created_at']).dt.dayofweek
    df['is_reply'] = df['in_reply_to_user_id'].notna().astype(int)
    df['hashtag_count'] = df['text'].apply(lambda x: len(re.findall(r'#\w+', x)))
    df['mention_count'] = df['text'].apply(lambda x: len(re.findall(r'@\w+', x)))
    df['retweet_like_ratio'] = df['retweet_count'] / (df['like_count'] + 1)
    df['sentiment'] = df['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
    df['subjectivity'] = df['text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    df['caps_ratio'] = df['text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)
    df['unique_words_ratio'] = df['text'].apply(lambda x: len(set(x.split())) / len(x.split()) if len(x.split()) > 0 else 0)
    df['tweet_frequency'] = df.groupby('author_id')['created_at'].transform('count')
    df['avg_word_length'] = df['text'].apply(lambda x: np.mean([len(word) for word in x.split()]) if len(x.split()) > 0 else 0)
    df['punctuation_count'] = df['text'].apply(lambda x: sum(1 for c in x if c in '.,;:!?'))
    df['emoji_count'] = df['text'].apply(lambda x: len(re.findall(r'[\U0001F600-\U0001F64F]', x)))
    df['exclamation_count'] = df['text'].apply(lambda x: x.count('!'))
    df['question_count'] = df['text'].apply(lambda x: x.count('?'))
    df['uppercase_word_count'] = df['text'].apply(lambda x: sum(1 for word in x.split() if word.isupper()))
    df['url_count'] = df['text'].apply(lambda x: len(re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', x)))
    df['digit_count'] = df['text'].apply(lambda x: sum(c.isdigit() for c in x))
    df['special_char_count'] = df['text'].apply(lambda x: len(re.findall(r'[^a-zA-Z0-9\s]', x)))
    df['word_count'] = df['text'].apply(lambda x: len(x.split()))
    df['unique_char_ratio'] = df['text'].apply(lambda x: len(set(x)) / len(x) if len(x) > 0 else 0)
    return df

def get_bert_embeddings(texts, model, tokenizer):
    embeddings = []
    for text in tqdm(texts):
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
    return np.array(embeddings)

class XGBoostProgressCallback(xgb.callback.TrainingCallback):
    def after_iteration(self, model, epoch, evals_log):
        if (epoch + 1) % 50 == 0 or (epoch + 1) == model.num_boosted_rounds():
            print(f'XGBoost iteration {epoch + 1}/{model.num_boosted_rounds()}')
        return False

print("データの読み込みと前処理を開始します...")
df = pd.read_csv('twibot_tweet0_new.csv')
df = extract_features(df)
print("前処理が完了しました。")

X_text = df['text']
numeric_features = [col for col in df.columns if col not in ['text', 'label', 'author_id', 'created_at', 'in_reply_to_user_id']]
X_numeric = df[numeric_features]
y = df['label'].map({'human': 0, 'bot': 1})

X_train_text, X_test_text, X_train_numeric, X_test_numeric, y_train, y_test = train_test_split(X_text, X_numeric, y, test_size=0.2, random_state=42, stratify=y)

tfidf = TfidfVectorizer(max_features=25000, ngram_range=(1, 3))
X_train_tfidf = tfidf.fit_transform(X_train_text)
X_test_tfidf = tfidf.transform(X_test_text)

scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.transform(X_test_numeric)

print("BERTモデルを使用して文章の埋め込みを生成します...")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
X_train_bert = get_bert_embeddings(X_train_text, model, tokenizer)
X_test_bert = get_bert_embeddings(X_test_text, model, tokenizer)

X_train_combined = np.hstack([X_train_tfidf.toarray(), X_train_numeric_scaled, X_train_bert])
X_test_combined = np.hstack([X_test_tfidf.toarray(), X_test_numeric_scaled, X_test_bert])

variance_threshold = VarianceThreshold(threshold=0.005)
X_train_var_selected = variance_threshold.fit_transform(X_train_combined)
X_test_var_selected = variance_threshold.transform(X_test_combined)

selector = SelectKBest(f_classif, k="")
X_train_selected = selector.fit_transform(X_train_var_selected, y_train)
X_test_selected = selector.transform(X_test_var_selected)

smote = SMOTE(sampling_strategy=0.7, random_state=42)
enn = EditedNearestNeighbours()
steps = [('over', smote), ('under', enn)]
pipeline = ImbPipeline(steps=steps)

X_train_resampled, y_train_resampled = pipeline.fit_resample(X_train_selected, y_train)

models = {
    'RandomForest': RandomForestClassifier(n_estimators=1000, max_depth=30, class_weight='balanced', random_state=42, n_jobs=-1),
    'XGBoost': xgb.XGBClassifier(n_estimators=1000, learning_rate=0.05, max_depth=7, random_state=42, callbacks=[XGBoostProgressCallback()], n_jobs=-1),
    'SVM': SVC(kernel='rbf', C=10, gamma='scale', probability=True, class_weight='balanced', random_state=42),
    'MLP': MLPClassifier(hidden_layer_sizes=(200, 100, 50), max_iter=1000, alpha=0.0001, random_state=42)
}

print("モデルの学習を開始します...")
for name, model in tqdm(models.items()):
    print(f"{name}の学習を開始します...")
    start_time = time.time()
    model.fit(X_train_resampled, y_train_resampled)
    end_time = time.time()
    print(f"{name}の学習が完了しました。学習時間: {end_time - start_time:.2f}秒")

print("アンサンブル予測を行います...")
y_pred_rf = models['RandomForest'].predict_proba(X_test_selected)[:, 1]
y_pred_xgb = models['XGBoost'].predict_proba(X_test_selected)[:, 1]
y_pred_svm = models['SVM'].predict_proba(X_test_selected)[:, 1]
y_pred_mlp = models['MLP'].predict_proba(X_test_selected)[:, 1]

y_pred_ensemble = 0.3 * y_pred_rf + 0.3 * y_pred_xgb + 0.2 * y_pred_svm + 0.2 * y_pred_mlp
y_pred_final = (y_pred_ensemble > 0.2).astype(int)  # 閾値を0.3から0.2に下げてボット検出を改善

print("最終的な予測結果:")
print(classification_report(y_test, y_pred_final, target_names=['human', 'bot']))


データの読み込みと前処理を開始します...
前処理が完了しました。
BERTモデルを使用して文章の埋め込みを生成します...


100%|███████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [01:50<00:00, 18.17it/s]


モデルの学習を開始します...


  0%|                                                                                                         | 0/4 [00:00<?, ?it/s]

RandomForestの学習を開始します...


...

 75%|████████████████████████████████████████████████████████████████████████▊                        | 3/4 [02:15<00:43, 43.43s/it]

SVMの学習が完了しました。学習時間: 35.59秒
MLPの学習を開始します...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:30<00:00, 37.57s/it]

MLPの学習が完了しました。学習時間: 14.92秒
アンサンブル予測を行います...


最終的な予測結果:
              precision    recall  f1-score   support

       human       0.97      0.12      0.21      1796
         bot       0.11      0.97      0.20       204

    accuracy                           0.21      2000
   macro avg       0.54      0.54      0.21      2000
weighted avg       0.88      0.21      0.21      2000

